In [1]:
#!pip install tensorboardcolab

In [2]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm

from torchvision.datasets import FashionMNIST
from torchvision import transforms

# 훈려용 데이터 가져오기
# 초기 상태에선 PIL（Python Imaging Library) 이미지 형식으로
# Dataset를 만들어 버린다.
# 따라서 transforms.ToTensor를 사용해 Tensor로 변환한다
fashion_mnist_train = FashionMNIST("data",
    train=True, download=True,
    transform=transforms.ToTensor())
# 검증용 데이터 가져오기
fashion_mnist_test = FashionMNIST("data",
    train=False, download=True,
    transform=transforms.ToTensor())

# 배치 크기가 128인 DataLoader를 각각 작성
batch_size=128
train_loader = DataLoader(fashion_mnist_train,
                          batch_size=batch_size, shuffle=True)
test_loader = DataLoader(fashion_mnist_test,
                          batch_size=batch_size, shuffle=False)

In [3]:
# (N, C, H, W)혀익의 Tensor를(N, C*H*W)로 늘리는 계층
# 합성곱 출력을 MLP에 전달할 때 필요
class FlattenLayer(nn.Module):
    def forward(self, x):
        sizes = x.size()
        return x.view(sizes[0], -1)

# 5×5의 커널을 사용해서 처음에 32개, 다음에 64개의 채널 작성
# BatchNorm2d는 이미지용 Batch Normalization
# Dropout2d는 이미지용 Dropout
# 마지막으로 FlattenLayer 적용
conv_net = nn.Sequential(
    nn.Conv2d(1, 32, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(0.25),
    nn.Conv2d(32, 64, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.25),
    FlattenLayer()
)

# 합성곱에 의해 최종적으로 이미지 크기가 어떤지를
# 더미 데이터를 넣어서 확인한다

test_input = torch.ones(1, 1, 28, 28)
conv_output_size = conv_net(test_input).size()[-1]

# 2층 MLP
mlp = nn.Sequential(
    nn.Linear(conv_output_size, 200),
    nn.ReLU(),
    nn.BatchNorm1d(200),
    nn.Dropout(0.25),
    nn.Linear(200, 10)
)
# 최종 CNN
net = nn.Sequential(
    conv_net,
    mlp
)

In [4]:
# 평가용 헬퍼 함수
def eval_net(net, data_loader, device="cpu"):
    # Dropout 및 BatchNorm을 무효화
    net.eval()
    ys = []
    ypreds = []
    for x, y in data_loader:
        # to 메서드로 계산을 실행할 디바이스로 전송
        x = x.to(device)
        y = y.to(device)
        # 확률이 가장 큰 클래스를 예측(리스트 2.1 참조)
        # 여기선 forward（추론） 계산이 전부이므로 자동 미분에
        # 필요한 처리는 off로 설정해서 불필요한 계산을 제한다
        with torch.no_grad():
            _, y_pred = net(x).max(1)
        ys.append(y)
        ypreds.append(y_pred)
    
    # 미니 배치 단위의 예측 결과 등을 하나로 묶는다
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    # 예측 정확도 계산
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

# 훈련용 헬퍼 함수
def train_net(net, train_loader, test_loader,
              optimizer_cls=optim.Adam,
              loss_fn=nn.CrossEntropyLoss(),
              n_iter=10, device="cpu", writer=None):
    train_losses = []
    train_acc = []
    val_acc = []
    optimizer = optimizer_cls(net.parameters())
    for epoch in range(n_iter):
        running_loss = 0.0
        # 신경망을 훈련 모드로 설정
        net.train()
        n = 0
        n_acc = 0
        # 시간이 많이 걸리므로 tqdm을 사용해서 진행바를 표시
        for i, (xx, yy) in tqdm.tqdm(enumerate(train_loader),
            total=len(train_loader)):
            xx = xx.to(device)
            yy = yy.to(device)
            h = net(xx)
            loss = loss_fn(h, yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += len(xx)
            _, y_pred = h.max(1)
            n_acc += (yy == y_pred).float().sum().item()
        train_losses.append(running_loss / i)
        # 훈련 데이터의 예측 정확도
        train_acc.append(n_acc / n)

        # 검증 데이터의 예측 정확도
        val_acc.append(eval_net(net, test_loader, device))
        # epoch의 결과 표시
        print(epoch, train_losses[-1], train_acc[-1],
            val_acc[-1], flush=True)
        if writer is not None:
            writer.add_scalar('train_loss', train_losses[-1], epoch)
            writer.add_scalars('accuracy', {
                "train": train_acc[-1],
                "validation": val_acc[-1]
            }, epoch)


In [5]:
from tensorboardX import SummaryWriter

# SummaryWriter 작성
writer = SummaryWriter("data/cnn")

# 훈련 실행
net.to("cuda:0")
train_net(net, train_loader, test_loader, n_iter=20, device="cuda:0", writer=writer)

100%|██████████| 469/469 [00:06<00:00, 72.00it/s]


0 0.47804910995257205 0.8339833333333333 0.87909996509552


100%|██████████| 469/469 [00:06<00:00, 73.15it/s]


1 0.32084448075192606 0.8825333333333333 0.8947999477386475


100%|██████████| 469/469 [00:06<00:00, 73.59it/s]


...

17 0.16009286854766372 0.9407166666666666 0.9222999811172485


100%|██████████| 469/469 [00:06<00:00, 74.49it/s]


18 0.15526436990461287 0.9418166666666666 0.9208999872207642


100%|██████████| 469/469 [00:06<00:00, 75.00it/s]


19 0.150135051092913 0.9435 0.9181999564170837


In [6]:
!tensorboard --logdir data/cnn

TensorBoard 1.13.1 at http://ubuntu16:6006 (Press CTRL+C to quit)
I0605 16:58:03.965746 139837908952832 _internal.py:122] ::ffff:127.0.0.1 - - [05/Jun/2019 16:58:03] "GET / HTTP/1.1" 200 -
I0605 16:58:04.536676 139837826922240 _internal.py:122] ::ffff:127.0.0.1 - - [05/Jun/2019 16:58:04] "GET /font-roboto/oMMgfZMQthOryQo9n22dcuvvDin1pK8aKteLpeZ5c0A.woff2 HTTP/1.1" 200 -
I0605 16:58:05.062269 139837908952832 _internal.py:122] ::ffff:127.0.0.1 - - [05/Jun/2019 16:58:05] "GET /tf-interactive-inference-dashboard/editedexample.png HTTP/1.1" 200 -
I0605 16:58:05.062673 139837826922240 _internal.py:122] ::ffff:127.0.0.1 - - [05/Jun/2019 16:58:05] "GET /tf-interactive-inference-dashboard/distance.png HTTP/1.1" 200 -
I0605 16:58:05.063395 139837818529536 _internal.py:122] ::ffff:127.0.0.1 - - [05/Jun/2019 16:58:05] "GET /tf-interactive-inference-dashboard/explorecounterfactuals.png HTTP/1.1" 200 -
I0605 16:58:05.065801 139837810136832 _internal.py:122] ::ffff:127.0.0.1 - - [05/Jun/2019 16:58:05